In [1]:
# Copyright (c) 2024 Byeonghyeon Kim 
# github site: https://github.com/bhkim003/ByeonghyeonKim
# email: bhkim003@snu.ac.kr
 
# Permission is hereby granted, free of charge, to any person obtaining a copy of
# this software and associated documentation files (the "Software"), to deal in
# the Software without restriction, including without limitation the rights to
# use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
# the Software, and to permit persons to whom the Software is furnished to do so,
# subject to the following conditions:
 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
# FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
# COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
# IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


In [2]:
# 메인 셀

import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

import time

from snntorch import spikegen
import matplotlib.pyplot as plt
import snntorch.spikeplot as splt
from IPython.display import HTML

# from torch.utils.tensorboard import SummaryWriter

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1,2,3,4,5"




class SYNAPSE_FC_METHOD(torch.autograd.Function):
    @staticmethod
    def forward(ctx, spike_one_time, spike_now, weight, bias):
        ctx.save_for_backward(spike_one_time, spike_now, weight, bias)
        return F.linear(spike_one_time, weight, bias=bias)

    @staticmethod
    def backward(ctx, grad_output_current):
        #############밑에부터 수정해라#######
        spike_one_time, spike_now, weight, bias = ctx.saved_tensors
        
        ## 이거 클론해야되는지 모르겠음!!!!
        grad_output_current_clone = grad_output_current.clone()

        grad_input_spike = grad_weight = grad_bias = None


        if ctx.needs_input_grad[0]:
            grad_input_spike = grad_output_current_clone @ weight
        if ctx.needs_input_grad[2]:
            grad_weight = grad_output_current_clone.t() @ spike_now
        if bias is not None and ctx.needs_input_grad[3]:
            grad_bias = grad_output_current_clone.sum(0)

        return grad_input_spike, None, grad_weight, grad_bias

     
class SYNAPSE_FC(nn.Module):
    def __init__(self, in_features, out_features, trace_const1=1, trace_const2=0.7):
        super(SYNAPSE_FC, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.trace_const1 = trace_const1
        self.trace_const2 = trace_const2

        # self.weight = torch.randn(self.out_features, self.in_features, requires_grad=True)
        # self.bias = torch.randn(self.out_features, requires_grad=True)
        self.weight = nn.Parameter(torch.randn(self.out_features, self.in_features))
        self.bias = nn.Parameter(torch.randn(self.out_features))

    def forward(self, spike):
        # spike: [Time, Batch, Features]   
        Time = spike.shape[0]
        Batch = spike.shape[1] 
        output_current = torch.zeros(Time, Batch, self.out_features, device=spike.device)

        # spike_detach = spike.detach().clone()
        spike_detach = spike.detach()
        spike_past = torch.zeros_like(spike_detach[0], device=spike.device)
        spike_now = torch.zeros_like(spike_detach[0], device=spike.device)

        for t in range(Time):
            spike_now = self.trace_const1*spike_detach[t] + self.trace_const2*spike_past
            output_current[t]= SYNAPSE_FC_METHOD.apply(spike[t], spike_now, self.weight, self.bias) 
            spike_past = spike_now

        return output_current 



class SYNAPSE_CONV_METHOD(torch.autograd.Function):
    @staticmethod
    def forward(ctx, spike_one_time, spike_now, weight, bias, stride=1, padding=1):
        ctx.save_for_backward(spike_one_time, spike_now, weight, bias, torch.tensor([stride], requires_grad=False), torch.tensor([padding], requires_grad=False))
        return F.conv2d(spike_one_time, weight, bias=bias, stride=stride, padding=padding)

    @staticmethod
    def backward(ctx, grad_output_current):
        spike_one_time, spike_now, weight, bias, stride, padding = ctx.saved_tensors
        stride=stride.item()
        padding=padding.item()
        
        ## 이거 클론해야되는지 모르겠음!!!!
        grad_output_current_clone = grad_output_current.clone()

        grad_input_spike = grad_weight = grad_bias = None


        if ctx.needs_input_grad[0]:
            grad_input_spike = F.conv_transpose2d(grad_output_current_clone, weight, stride=stride, padding=padding)
        if ctx.needs_input_grad[2]:
            grad_weight = torch.nn.grad.conv2d_weight(spike_now, weight.shape, grad_output_current_clone,
                                                      stride=stride, padding=padding)
        if bias is not None and ctx.needs_input_grad[3]:
            grad_bias = grad_output_current_clone.sum((0, -1, -2))

        return grad_input_spike, None, grad_weight, grad_bias, None, None

     



class SYNAPSE_CONV(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, trace_const1=1, trace_const2=0.7):
        super(SYNAPSE_CONV, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.trace_const1 = trace_const1
        self.trace_const2 = trace_const2

        # self.weight = torch.randn(self.out_channels, self.in_channels, self.kernel_size, self.kernel_size, requires_grad=True)
        # self.bias = torch.randn(self.out_channels, requires_grad=True)
        self.weight = nn.Parameter(torch.randn(self.out_channels, self.in_channels, self.kernel_size, self.kernel_size))
        self.bias = nn.Parameter(torch.randn(self.out_channels))

    def forward(self, spike):
        # spike: [Time, Batch, Channel, Height, Width]   
        # print('spike.shape', spike.shape)
        Time = spike.shape[0]
        Batch = spike.shape[1] 
        Channel = self.out_channels
        Height = (spike.shape[3] + self.padding*2 - self.kernel_size) // self.stride + 1
        Width = (spike.shape[4] + self.padding*2 - self.kernel_size) // self.stride + 1
        output_current = torch.zeros(Time, Batch, Channel, Height, Width, device=spike.device)

        # spike_detach = spike.detach().clone()
        spike_detach = spike.detach()
        spike_past = torch.zeros_like(spike_detach[0])
        spike_now = torch.zeros_like(spike_detach[0])

        for t in range(Time):
            spike_now = self.trace_const1*spike_detach[t] + self.trace_const2*spike_past
            output_current[t]= SYNAPSE_CONV_METHOD.apply(spike[t], spike_now, self.weight, self.bias, self.stride, self.padding) 
            spike_past = spike_now

        return output_current



class LIF_METHOD(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input_current_one_time, v_one_time, v_decay, v_threshold, v_reset, sg_width):
        v_one_time = v_one_time * v_decay + input_current_one_time # leak + pre-synaptic current integrate
        spike = (v_one_time >= v_threshold).float() #fire
        ctx.save_for_backward(v_one_time, torch.tensor([v_decay], requires_grad=False), 
                              torch.tensor([v_threshold], requires_grad=False), 
                              torch.tensor([v_reset], requires_grad=False), 
                              torch.tensor([sg_width], requires_grad=False)) # save before reset
        v_one_time = (v_one_time - spike * v_threshold).clamp_min(0) # reset
        return spike, v_one_time

    @staticmethod
    def backward(ctx, grad_output_spike, grad_output_v):
        v_one_time, v_decay, v_threshold, v_reset, sg_width = ctx.saved_tensors
        v_decay=v_decay.item()
        v_threshold=v_threshold.item()
        v_reset=v_reset.item()
        sg_width=sg_width.item()

        grad_input_current = grad_output_spike.clone()
        # grad_temp_v = grad_output_v.clone() # not used

        ################ select one of the following surrogate gradient functions ################
        #===========surrogate gradient function (rectangle)
        grad_input_current = grad_input_current * ((v_one_time - v_threshold).abs() < sg_width/2).float() / sg_width

        #===========surrogate gradient function (sigmoid)
        # sig = torch.sigmoid((v_one_time - v_threshold))
        # grad_input_current =  sig*(1-sig)*grad_input_current

        #===========surrogate gradient function (rough rectangle)
        # v_minus_th = (v_one_time - v_threshold)
        # grad_input_current[v_minus_th <= -.5] = 0
        # grad_input_current[v_minus_th > .5] = 0
        ###########################################################################################
        return grad_input_current, None, None, None, None, None

class LIF_layer(nn.Module):
    def __init__ (self, v_init = 0.0, v_decay = 0.8, v_threshold = 0.5, v_reset = 0.0, sg_width = 1):
        super(LIF_layer, self).__init__()
        self.v_init = v_init
        self.v_decay = v_decay
        self.v_threshold = v_threshold
        self.v_reset = v_reset
        self.sg_width = sg_width

    def forward(self, input_current):
        v = torch.full_like(input_current, fill_value = self.v_init, dtype = torch.float) # v (membrane potential) init
        post_spike = torch.full_like(input_current, fill_value = self.v_init, device=input_current.device, dtype = torch.float) # v (membrane potential) init
        # i와 v와 post_spike size는 여기서 다 같음: [Time, Batch, Channel, Height, Width] 

        Time = v.shape[0]
        for t in range(Time):
            # leaky하고 input_current 더하고 fire하고 reset까지 (backward직접처리)
            post_spike[t], v[t] = LIF_METHOD.apply(input_current[t], v[t], 
                                            self.v_decay, self.v_threshold, self.v_reset, self.sg_width) 

        return post_spike   
    
    


torch.manual_seed(42)








In [3]:
# HEPER PARAMETER
TIME = 8
BATCH = 256
# IMAGE_PIXEL_CHANNEL = 3
IMAGE_SIZE = 32
# CLASS_NUM = 10

## 데이터셋 선택하세요 #########
# which_data = 'MNIST'
which_data = 'CIFAR10'
################################

rate_coding = True

if (which_data == 'MNIST'):
    data_path = '/data2'

    if rate_coding :
        transform = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                    transforms.ToTensor()])
    else : 
        transform = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5))])

    trainset = torchvision.datasets.MNIST(root=data_path,
                                        train=True,
                                        download=True,
                                        transform=transform)


    testset = torchvision.datasets.MNIST(root=data_path,
                                        train=False,
                                        download=True,
                                        transform=transform)

    train_loader = DataLoader(trainset,
                            batch_size =BATCH,
                            shuffle = True,
                            num_workers =2)
    test_loader = DataLoader(testset,
                            batch_size =BATCH,
                            shuffle = False,
                            num_workers =2)


if (which_data == 'CIFAR10'):
    data_path = '/data2/cifar10'

    if rate_coding :
        transform_train = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor()])

        transform_test = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                            transforms.ToTensor()])
    
    else :
        transform_train = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

        transform_test = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),])


    trainset = torchvision.datasets.CIFAR10(root=data_path,
                                        train=True,
                                        download=True,
                                        transform=transform_train)


    testset = torchvision.datasets.CIFAR10(root=data_path,
                                        train=False,
                                        download=True,
                                        transform=transform_test)

    train_loader = DataLoader(trainset,
                            batch_size =BATCH,
                            shuffle = True,
                            num_workers =2)
    test_loader = DataLoader(testset,
                            batch_size =BATCH,
                            shuffle = False,
                            num_workers =2)

    '''
    classes = ('plane', 'car', 'bird', 'cat', 'deer',
            'dog', 'frog', 'horse', 'ship', 'truck') 
    '''


# 데이터 로더에서 첫 번째 배치를 가져옵니다.
data_iter = iter(train_loader)
images, labels = data_iter.next()

# 채널 수와 클래스 개수를 확인합니다.
IMAGE_PIXEL_CHANNEL = images.shape[1]
CLASS_NUM = len(torch.unique(labels))
print('IMAGE_PIXEL_CHANNEL:', IMAGE_PIXEL_CHANNEL)
print('CLASS_NUM:', CLASS_NUM)
print(images.shape)

Files already downloaded and verified
Files already downloaded and verified
IMAGE_PIXEL_CHANNEL: 3
CLASS_NUM: 10
torch.Size([256, 3, 32, 32])


In [4]:
## LIF_layer 레이어의 하이퍼파라미터
lif_layer_v_init = 0.0
lif_layer_v_decay = 0.6 
lif_layer_v_threshold = 1.2
lif_layer_v_reset = 0.0
lif_layer_sg_width = 1

## SYNAPSE_CONV 레이어의 하이퍼파라미터
synapse_conv_in_channels = IMAGE_PIXEL_CHANNEL
# synapse_conv_out_channels = layer별 지정
synapse_conv_kernel_size = 3
synapse_conv_stride = 1
synapse_conv_padding = 1
synapse_conv_trace_const1 = 1
synapse_conv_trace_const2 = lif_layer_v_decay

## SYNAPSE_FC 레이어의 하이퍼파라미터
# synapse_fc_in_features = 마지막CONV_OUT_CHANNEL * H * W
synapse_fc_out_features = CLASS_NUM
synapse_fc_trace_const1 = 1
synapse_fc_trace_const2 = lif_layer_v_decay


def make_layers_conv(cfg, in_c=3):
    layers = []
    img_size_var = IMAGE_SIZE
    in_channels = in_c
    for which in cfg:
        if which == 'P':
            # layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            layers += [nn.AvgPool2d(kernel_size=2, stride=2)]
            img_size_var = img_size_var // 2
        else:
            out_channels = which
            layers += [SYNAPSE_CONV(in_channels=in_channels,
                                    out_channels=out_channels, 
                                    kernel_size=synapse_conv_kernel_size, 
                                    stride=synapse_conv_stride, 
                                    padding=synapse_conv_padding, 
                                    trace_const1=synapse_conv_trace_const1, 
                                    trace_const2=synapse_conv_trace_const2)]
            img_size_var = (img_size_var - synapse_conv_kernel_size + 2*synapse_conv_padding)//synapse_conv_stride + 1

            layers += [LIF_layer(v_init=lif_layer_v_init, 
                                    v_decay=lif_layer_v_decay, 
                                    v_threshold=lif_layer_v_threshold, 
                                    v_reset=lif_layer_v_reset, 
                                    sg_width=lif_layer_sg_width)]
            in_channels = which

    return nn.Sequential(*layers), in_channels, img_size_var



class MY_SNN_CONV(nn.Module):
    def __init__(self, cfg, inc = 3):
        super(MY_SNN_CONV, self).__init__()

        self.layers, self.conv_last_channels, self.img_size_var = make_layers_conv(cfg, in_c=inc)

        self.synapse_FC = SYNAPSE_FC(in_features=self.conv_last_channels*self.img_size_var*self.img_size_var,  # 마지막CONV의 OUT_CHANNEL * H * W
                                      out_features=synapse_fc_out_features, 
                                      trace_const1=synapse_fc_trace_const1, 
                                      trace_const2=synapse_fc_trace_const2)
        

    def forward(self, spike_input):
        # inputs: [Batch, Time, Channel, Height, Width]   
        spike_input = spike_input.permute(1, 0, 2, 3, 4)
        # inputs: [Time, Batch, Channel, Height, Width]   
        spike_input = self.layers(spike_input)

        spike_input = spike_input.view(spike_input.size(0), spike_input.size(1), -1)
        
        spike_input = self.synapse_FC(spike_input)

        spike_input = spike_input.sum(axis=0)

        return spike_input



def make_layers_fc(cfg, in_c=3, img_size = 32, out_c=10):
    layers = []
    in_channels = in_c * img_size * img_size
    class_num = out_c
    for which in cfg:
        out_channels = which
        layers += [SYNAPSE_FC(in_features=in_channels,  # 마지막CONV의 OUT_CHANNEL * H * W
                                      out_features=out_channels, 
                                      trace_const1=synapse_fc_trace_const1, 
                                      trace_const2=synapse_fc_trace_const2)]

        layers += [LIF_layer(v_init=lif_layer_v_init, 
                                v_decay=lif_layer_v_decay, 
                                v_threshold=lif_layer_v_threshold, 
                                v_reset=lif_layer_v_reset, 
                                sg_width=lif_layer_sg_width)]
        in_channels = which

    
    out_channels = class_num
    layers += [SYNAPSE_FC(in_features=in_channels,  
                                    out_features=out_channels, 
                                    trace_const1=synapse_fc_trace_const1, 
                                    trace_const2=synapse_fc_trace_const2)]
    
    return nn.Sequential(*layers)

class MY_SNN_FC(nn.Module):
    def __init__(self, cfg, inc = 3):
        super(MY_SNN_FC, self).__init__()

        self.layers = make_layers_fc(cfg, in_c=inc, img_size = IMAGE_SIZE, out_c=synapse_fc_out_features)

        

    def forward(self, spike_input):
        # inputs: [Batch, Time, Channel, Height, Width]   
        spike_input = spike_input.permute(1, 0, 2, 3, 4)
        # inputs: [Time, Batch, Channel, Height, Width]   
        spike_input = spike_input.view(spike_input.size(0), spike_input.size(1), -1)
        
        spike_input = self.layers(spike_input)

        spike_input = spike_input.sum(axis=0)

        return spike_input
    


In [5]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

#########################
##### net 선택 #############

cfg = {
    'A': [64, 64], 
    'B': [64, 64, 64, 64], 
    'C': [64, 128, 256],
    'D': [64, 128, 'P', 256, 256, 'P', 512, 512, 'P', 512, 512],
    'K': [64, 64],
}

#### 새로 net 만들기 ####
which_cfg = 'D'

if (which_cfg >= 'K'):
    net = MY_SNN_FC(cfg[which_cfg], inc = synapse_conv_in_channels).to(device)
else:
    net = MY_SNN_CONV(cfg[which_cfg], inc = synapse_conv_in_channels).to(device)
net = torch.nn.DataParallel(net)
val_acc = 0
########################

#### model 저장해놨던거 쓰기##############
net = torch.load("net_save/save_now_net.pth")
# net = torch.load("net_save/mnist97.pth")
##########################################

learning_rate = 0.0001

net = net.to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
# optimizer = torch.optim.Adam(net.parameters(), lr=0.00001)


for epoch in range(100):
    print('epoch', epoch)
    epoch_start_time = time.time()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        net.train()

        # print('\niter', i)
        iter_one_train_time_start = time.time()

        inputs, labels = data

        if rate_coding == True :
            inputs = spikegen.rate(inputs, num_steps=TIME)
        else :
            inputs = inputs.repeat(TIME, 1, 1, 1, 1)
        
        # inputs: [Time, Batch, Channel, Height, Width]   

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # inputs: [Time, Batch, Channel, Height, Width]   
        inputs = inputs.permute(1, 0, 2, 3, 4) # net에 넣어줄때는 batch가 젤 앞 차원으로 와야함.
        # inputs: [Batch, Time, Channel, Height, Width]   


        outputs = net(inputs)

        batch = BATCH 
        if labels.size(0) != BATCH: 
            batch = labels.size(0)


        ####### training accruacy ######
        correct = 0
        total = 0
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted[0:batch] == labels).sum().item()
        if i % 100 == 9:
            print(f'training acc: {100 * correct / total:.2f}%')
        ################################

        loss = criterion(outputs[0:batch,:], labels)
        loss.backward()

        # optimizer.zero_grad()와 loss.backward() 호출 후에 실행해야 합니다.
        # if (i % 100 == 9):
        #     print('\n\nepoch', epoch, 'iter', i)
        #     for name, param in net.named_parameters():
        #         if param.requires_grad:
        #             print('\n\n\n\n' , name, param.grad)
        
        optimizer.step()

        running_loss += loss.item()
        # print("Epoch: {}, Iter: {}, Loss: {}".format(epoch + 1, i + 1, running_loss / 100))

        iter_one_train_time_end = time.time()
        elapsed_time = iter_one_train_time_end - iter_one_train_time_start  # 실행 시간 계산
        # print(f"iter_one_train_time: {elapsed_time} seconds")


        if i % 100 == 9:
            iter_one_val_time_start = time.time()
            
            correct = 0
            total = 0
            with torch.no_grad():
                net.eval()
                how_many_val_image=0
                for data in test_loader:
                    how_many_val_image += 1
                    inputs, labels = data
        
                    if rate_coding == True :
                        inputs = spikegen.rate(inputs, num_steps=TIME)
                    else :
                        inputs = inputs.repeat(TIME, 1, 1, 1, 1)

                    
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    outputs = net(inputs.permute(1, 0, 2, 3, 4))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    batch = BATCH 
                    if labels.size(0) != BATCH: 
                        batch = labels.size(0)
                    correct += (predicted[0:batch] == labels).sum().item()
                print(f'validation acc: {100 * correct / total:.2f}%')


            iter_one_val_time_end = time.time()
            elapsed_time = iter_one_val_time_end - iter_one_val_time_start  # 실행 시간 계산
            print(f"iter_one_val_time: {elapsed_time} seconds")
            if val_acc < correct / total:
                val_acc = correct / total
                torch.save(net.state_dict(), "net_save/save_now_net_weights.pth")
                torch.save(net, "net_save/save_now_net.pth")
                torch.save(net.module.state_dict(), "net_save/save_now_net_weights2.pth")
                torch.save(net.module, "net_save/save_now_net2.pth")
    epoch_time_end = time.time()
    epoch_time = epoch_time_end - epoch_start_time  # 실행 시간 계산
    
    print(f"epoch_time: {epoch_time} seconds")
    print('\n')


epoch 0
training acc: 15.62%
validation acc: 19.11%
iter_one_val_time: 5.670857667922974 seconds
training acc: 19.92%
validation acc: 11.95%
iter_one_val_time: 5.922300815582275 seconds
epoch_time: 79.34645533561707 seconds


epoch 1
training acc: 15.23%
validation acc: 16.85%
iter_one_val_time: 5.420670032501221 seconds
training acc: 14.45%
validation acc: 14.13%
iter_one_val_time: 5.785154104232788 seconds
epoch_time: 62.71440100669861 seconds


epoch 2
training acc: 11.33%
validation acc: 14.35%
iter_one_val_time: 5.680572032928467 seconds
training acc: 10.16%
validation acc: 14.70%
iter_one_val_time: 5.542245626449585 seconds
epoch_time: 61.480024337768555 seconds


epoch 3
training acc: 13.67%
validation acc: 13.54%
iter_one_val_time: 5.651238679885864 seconds
training acc: 13.67%
validation acc: 13.40%
iter_one_val_time: 5.23375129699707 seconds
epoch_time: 61.69606018066406 seconds


epoch 4
training acc: 17.19%
validation acc: 17.43%
iter_one_val_time: 5.714173078536987 seconds

In [ ]:
torch.manual_seed(42)

data_path = '/data2'

transform = transforms.Compose([transforms.Resize((28, 28)),
                            transforms.ToTensor()])

# OR

transform = transforms.Compose([transforms.Resize((28, 28)),
                                transforms.Lambda(lambda x: torch.from_numpy(np.array(x)))])

trainset = torchvision.datasets.MNIST(root=data_path,
                                        train=True,
                                        download=True,
                                        transform=transform)


train_loader = DataLoader(trainset,
                        batch_size =64,
                        shuffle = True,
                        num_workers =2)

from snntorch import spikegen
torch.set_printoptions(threshold=torch.inf)

# Iterate through minibatches
data = iter(train_loader)
data_it, targets_it = next(data)
# Spiking Data
# data_it = spikegen.rate(data_it, num_steps=8)
print(data_it.shape)

# print(data_it[0,0,0,:,:])

# print(data_it[0])
# print(targets_it[0])


a = torch.Tensor([1, .2, .7, 1])
a=spikegen.rate(a, num_steps=8)
a

torch.Size([64, 28, 28])


tensor([[1., 0., 1., 1.],
        [1., 1., 0., 1.],
        [1., 0., 0., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 1., 0., 1.],
        [1., 0., 1., 1.]])

In [ ]:
net

DataParallel(
  (module): MY_SNN_CONV(
    (layers): Sequential(
      (0): SYNAPSE_CONV()
      (1): LIF_layer()
      (2): SYNAPSE_CONV()
      (3): LIF_layer()
    )
    (synapse_FC): SYNAPSE_FC()
  )
)

In [ ]:
def my_snn_system(devices = "0,1,2,3",
                    my_seed = 42,
                    TIME = 8,
                    BATCH = 256,
                    IMAGE_SIZE = 32,
                    which_data = 'CIFAR10',
                    rate_coding = True,
    
                    lif_layer_v_init = 0.0,
                    lif_layer_v_decay = 0.6,
                    lif_layer_v_threshold = 1.2,
                    lif_layer_v_reset = 0.0,
                    lif_layer_sg_width = 1,

                    synapse_conv_in_channels = IMAGE_PIXEL_CHANNEL,
                    synapse_conv_kernel_size = 3,
                    synapse_conv_stride = 1,
                    synapse_conv_padding = 1,
                    synapse_conv_trace_const1 = 1,
                    synapse_conv_trace_const2 = lif_layer_v_decay,

                    synapse_fc_out_features = CLASS_NUM,
                    synapse_fc_trace_const1 = 1,
                    synapse_fc_trace_const2 = lif_layer_v_decay,

                    pre_trained = False,
                    convTrue_fcFalse = True,
                    cfg = [64, 64],
                    pre_trained_path = "net_save/save_now_net.pth",
                    learning_rate = 0.0001,
                    epoch_num = 100,
                    verbose_interval = 100
                  ):
    
    import sys
    import os
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torch.backends.cudnn as cudnn

    import torch.optim as optim
    from torch.utils.data import Dataset, DataLoader

    import torchvision
    import torchvision.datasets
    import torchvision.transforms as transforms

    import numpy as np
    import matplotlib.pyplot as plt

    import time

    from snntorch import spikegen
    import matplotlib.pyplot as plt
    import snntorch.spikeplot as splt
    from IPython.display import HTML

    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
    os.environ["CUDA_VISIBLE_DEVICES"]= devices

    
    torch.manual_seed(my_seed)


    class SYNAPSE_FC_METHOD(torch.autograd.Function):
        @staticmethod
        def forward(ctx, spike_one_time, spike_now, weight, bias):
            ctx.save_for_backward(spike_one_time, spike_now, weight, bias)
            return F.linear(spike_one_time, weight, bias=bias)

        @staticmethod
        def backward(ctx, grad_output_current):
            #############밑에부터 수정해라#######
            spike_one_time, spike_now, weight, bias = ctx.saved_tensors
            
            ## 이거 클론해야되는지 모르겠음!!!!
            grad_output_current_clone = grad_output_current.clone()

            grad_input_spike = grad_weight = grad_bias = None


            if ctx.needs_input_grad[0]:
                grad_input_spike = grad_output_current_clone @ weight
            if ctx.needs_input_grad[2]:
                grad_weight = grad_output_current_clone.t() @ spike_now
            if bias is not None and ctx.needs_input_grad[3]:
                grad_bias = grad_output_current_clone.sum(0)

            return grad_input_spike, None, grad_weight, grad_bias

     
    class SYNAPSE_FC(nn.Module):
        def __init__(self, in_features, out_features, trace_const1=1, trace_const2=0.7):
            super(SYNAPSE_FC, self).__init__()
            self.in_features = in_features
            self.out_features = out_features
            self.trace_const1 = trace_const1
            self.trace_const2 = trace_const2

            # self.weight = torch.randn(self.out_features, self.in_features, requires_grad=True)
            # self.bias = torch.randn(self.out_features, requires_grad=True)
            self.weight = nn.Parameter(torch.randn(self.out_features, self.in_features))
            self.bias = nn.Parameter(torch.randn(self.out_features))

        def forward(self, spike):
            # spike: [Time, Batch, Features]   
            Time = spike.shape[0]
            Batch = spike.shape[1] 
            output_current = torch.zeros(Time, Batch, self.out_features, device=spike.device)

            # spike_detach = spike.detach().clone()
            spike_detach = spike.detach()
            spike_past = torch.zeros_like(spike_detach[0], device=spike.device)
            spike_now = torch.zeros_like(spike_detach[0], device=spike.device)

            for t in range(Time):
                spike_now = self.trace_const1*spike_detach[t] + self.trace_const2*spike_past
                output_current[t]= SYNAPSE_FC_METHOD.apply(spike[t], spike_now, self.weight, self.bias) 
                spike_past = spike_now

            return output_current 



    class SYNAPSE_CONV_METHOD(torch.autograd.Function):
        @staticmethod
        def forward(ctx, spike_one_time, spike_now, weight, bias, stride=1, padding=1):
            ctx.save_for_backward(spike_one_time, spike_now, weight, bias, torch.tensor([stride], requires_grad=False), torch.tensor([padding], requires_grad=False))
            return F.conv2d(spike_one_time, weight, bias=bias, stride=stride, padding=padding)

        @staticmethod
        def backward(ctx, grad_output_current):
            spike_one_time, spike_now, weight, bias, stride, padding = ctx.saved_tensors
            stride=stride.item()
            padding=padding.item()
            
            ## 이거 클론해야되는지 모르겠음!!!!
            grad_output_current_clone = grad_output_current.clone()

            grad_input_spike = grad_weight = grad_bias = None


            if ctx.needs_input_grad[0]:
                grad_input_spike = F.conv_transpose2d(grad_output_current_clone, weight, stride=stride, padding=padding)
            if ctx.needs_input_grad[2]:
                grad_weight = torch.nn.grad.conv2d_weight(spike_now, weight.shape, grad_output_current_clone,
                                                        stride=stride, padding=padding)
            if bias is not None and ctx.needs_input_grad[3]:
                grad_bias = grad_output_current_clone.sum((0, -1, -2))

            return grad_input_spike, None, grad_weight, grad_bias, None, None

        



    class SYNAPSE_CONV(nn.Module):
        def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, trace_const1=1, trace_const2=0.7):
            super(SYNAPSE_CONV, self).__init__()
            self.in_channels = in_channels
            self.out_channels = out_channels
            self.kernel_size = kernel_size
            self.stride = stride
            self.padding = padding
            self.trace_const1 = trace_const1
            self.trace_const2 = trace_const2

            # self.weight = torch.randn(self.out_channels, self.in_channels, self.kernel_size, self.kernel_size, requires_grad=True)
            # self.bias = torch.randn(self.out_channels, requires_grad=True)
            self.weight = nn.Parameter(torch.randn(self.out_channels, self.in_channels, self.kernel_size, self.kernel_size))
            self.bias = nn.Parameter(torch.randn(self.out_channels))

        def forward(self, spike):
            # spike: [Time, Batch, Channel, Height, Width]   
            # print('spike.shape', spike.shape)
            Time = spike.shape[0]
            Batch = spike.shape[1] 
            Channel = self.out_channels
            Height = (spike.shape[3] + self.padding*2 - self.kernel_size) // self.stride + 1
            Width = (spike.shape[4] + self.padding*2 - self.kernel_size) // self.stride + 1
            output_current = torch.zeros(Time, Batch, Channel, Height, Width, device=spike.device)

            # spike_detach = spike.detach().clone()
            spike_detach = spike.detach()
            spike_past = torch.zeros_like(spike_detach[0])
            spike_now = torch.zeros_like(spike_detach[0])

            for t in range(Time):
                spike_now = self.trace_const1*spike_detach[t] + self.trace_const2*spike_past
                output_current[t]= SYNAPSE_CONV_METHOD.apply(spike[t], spike_now, self.weight, self.bias, self.stride, self.padding) 
                spike_past = spike_now

            return output_current



    class LIF_METHOD(torch.autograd.Function):
        @staticmethod
        def forward(ctx, input_current_one_time, v_one_time, v_decay, v_threshold, v_reset, sg_width):
            v_one_time = v_one_time * v_decay + input_current_one_time # leak + pre-synaptic current integrate
            spike = (v_one_time >= v_threshold).float() #fire
            ctx.save_for_backward(v_one_time, torch.tensor([v_decay], requires_grad=False), 
                                torch.tensor([v_threshold], requires_grad=False), 
                                torch.tensor([v_reset], requires_grad=False), 
                                torch.tensor([sg_width], requires_grad=False)) # save before reset
            v_one_time = (v_one_time - spike * v_threshold).clamp_min(0) # reset
            return spike, v_one_time

        @staticmethod
        def backward(ctx, grad_output_spike, grad_output_v):
            v_one_time, v_decay, v_threshold, v_reset, sg_width = ctx.saved_tensors
            v_decay=v_decay.item()
            v_threshold=v_threshold.item()
            v_reset=v_reset.item()
            sg_width=sg_width.item()

            grad_input_current = grad_output_spike.clone()
            # grad_temp_v = grad_output_v.clone() # not used

            ################ select one of the following surrogate gradient functions ################
            #===========surrogate gradient function (rectangle)
            grad_input_current = grad_input_current * ((v_one_time - v_threshold).abs() < sg_width/2).float() / sg_width

            #===========surrogate gradient function (sigmoid)
            # sig = torch.sigmoid((v_one_time - v_threshold))
            # grad_input_current =  sig*(1-sig)*grad_input_current

            #===========surrogate gradient function (rough rectangle)
            # v_minus_th = (v_one_time - v_threshold)
            # grad_input_current[v_minus_th <= -.5] = 0
            # grad_input_current[v_minus_th > .5] = 0
            ###########################################################################################
            return grad_input_current, None, None, None, None, None

    class LIF_layer(nn.Module):
        def __init__ (self, v_init = 0.0, v_decay = 0.8, v_threshold = 0.5, v_reset = 0.0, sg_width = 1):
            super(LIF_layer, self).__init__()
            self.v_init = v_init
            self.v_decay = v_decay
            self.v_threshold = v_threshold
            self.v_reset = v_reset
            self.sg_width = sg_width

        def forward(self, input_current):
            v = torch.full_like(input_current, fill_value = self.v_init, dtype = torch.float) # v (membrane potential) init
            post_spike = torch.full_like(input_current, fill_value = self.v_init, device=input_current.device, dtype = torch.float) # v (membrane potential) init
            # i와 v와 post_spike size는 여기서 다 같음: [Time, Batch, Channel, Height, Width] 

            Time = v.shape[0]
            for t in range(Time):
                # leaky하고 input_current 더하고 fire하고 reset까지 (backward직접처리)
                post_spike[t], v[t] = LIF_METHOD.apply(input_current[t], v[t], 
                                                self.v_decay, self.v_threshold, self.v_reset, self.sg_width) 

            return post_spike 















    if (which_data == 'MNIST'):
        data_path = '/data2'

        if rate_coding :
            transform = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                        transforms.ToTensor()])
        else : 
            transform = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5,),(0.5))])

        trainset = torchvision.datasets.MNIST(root=data_path,
                                            train=True,
                                            download=True,
                                            transform=transform)


        testset = torchvision.datasets.MNIST(root=data_path,
                                            train=False,
                                            download=True,
                                            transform=transform)

        train_loader = DataLoader(trainset,
                                batch_size =BATCH,
                                shuffle = True,
                                num_workers =2)
        test_loader = DataLoader(testset,
                                batch_size =BATCH,
                                shuffle = False,
                                num_workers =2)


    if (which_data == 'CIFAR10'):
        data_path = '/data2/cifar10'

        if rate_coding :
            transform_train = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                                transforms.RandomHorizontalFlip(),
                                                transforms.ToTensor()])

            transform_test = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                                transforms.ToTensor()])
        
        else :
            transform_train = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                                transforms.RandomHorizontalFlip(),
                                                transforms.ToTensor(),
                                                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

            transform_test = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                                transforms.ToTensor(),
                                                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),])


        trainset = torchvision.datasets.CIFAR10(root=data_path,
                                            train=True,
                                            download=True,
                                            transform=transform_train)


        testset = torchvision.datasets.CIFAR10(root=data_path,
                                            train=False,
                                            download=True,
                                            transform=transform_test)

        train_loader = DataLoader(trainset,
                                batch_size =BATCH,
                                shuffle = True,
                                num_workers =2)
        test_loader = DataLoader(testset,
                                batch_size =BATCH,
                                shuffle = False,
                                num_workers =2)

        '''
        classes = ('plane', 'car', 'bird', 'cat', 'deer',
                'dog', 'frog', 'horse', 'ship', 'truck') 
        '''
        













    def make_layers_conv(cfg, in_c=3):
        layers = []
        img_size_var = IMAGE_SIZE
        in_channels = in_c
        for which in cfg:
            if which == 'P':
                # layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
                layers += [nn.AvgPool2d(kernel_size=2, stride=2)]
                img_size_var = img_size_var // 2
            else:
                out_channels = which
                layers += [SYNAPSE_CONV(in_channels=in_channels,
                                        out_channels=out_channels, 
                                        kernel_size=synapse_conv_kernel_size, 
                                        stride=synapse_conv_stride, 
                                        padding=synapse_conv_padding, 
                                        trace_const1=synapse_conv_trace_const1, 
                                        trace_const2=synapse_conv_trace_const2)]
                img_size_var = (img_size_var - synapse_conv_kernel_size + 2*synapse_conv_padding)//synapse_conv_stride + 1

                layers += [LIF_layer(v_init=lif_layer_v_init, 
                                        v_decay=lif_layer_v_decay, 
                                        v_threshold=lif_layer_v_threshold, 
                                        v_reset=lif_layer_v_reset, 
                                        sg_width=lif_layer_sg_width)]
                in_channels = which

        return nn.Sequential(*layers), in_channels, img_size_var



    class MY_SNN_CONV(nn.Module):
        def __init__(self, cfg, inc = 3):
            super(MY_SNN_CONV, self).__init__()

            self.layers, self.conv_last_channels, self.img_size_var = make_layers_conv(cfg, in_c=inc)

            self.synapse_FC = SYNAPSE_FC(in_features=self.conv_last_channels*self.img_size_var*self.img_size_var,  # 마지막CONV의 OUT_CHANNEL * H * W
                                        out_features=synapse_fc_out_features, 
                                        trace_const1=synapse_fc_trace_const1, 
                                        trace_const2=synapse_fc_trace_const2)
            

        def forward(self, spike_input):
            # inputs: [Batch, Time, Channel, Height, Width]   
            spike_input = spike_input.permute(1, 0, 2, 3, 4)
            # inputs: [Time, Batch, Channel, Height, Width]   
            spike_input = self.layers(spike_input)

            spike_input = spike_input.view(spike_input.size(0), spike_input.size(1), -1)
            
            spike_input = self.synapse_FC(spike_input)

            spike_input = spike_input.sum(axis=0)

            return spike_input



    def make_layers_fc(cfg, in_c=3, img_size = 32, out_c=10):
        layers = []
        in_channels = in_c * img_size * img_size
        class_num = out_c
        for which in cfg:
            out_channels = which
            layers += [SYNAPSE_FC(in_features=in_channels,  # 마지막CONV의 OUT_CHANNEL * H * W
                                        out_features=out_channels, 
                                        trace_const1=synapse_fc_trace_const1, 
                                        trace_const2=synapse_fc_trace_const2)]

            layers += [LIF_layer(v_init=lif_layer_v_init, 
                                    v_decay=lif_layer_v_decay, 
                                    v_threshold=lif_layer_v_threshold, 
                                    v_reset=lif_layer_v_reset, 
                                    sg_width=lif_layer_sg_width)]
            in_channels = which

        
        out_channels = class_num
        layers += [SYNAPSE_FC(in_features=in_channels,  
                                        out_features=out_channels, 
                                        trace_const1=synapse_fc_trace_const1, 
                                        trace_const2=synapse_fc_trace_const2)]
        
        return nn.Sequential(*layers)

    class MY_SNN_FC(nn.Module):
        def __init__(self, cfg, inc = 3):
            super(MY_SNN_FC, self).__init__()

            self.layers = make_layers_fc(cfg, in_c=inc, img_size = IMAGE_SIZE, out_c=synapse_fc_out_features)

            

        def forward(self, spike_input):
            # inputs: [Batch, Time, Channel, Height, Width]   
            spike_input = spike_input.permute(1, 0, 2, 3, 4)
            # inputs: [Time, Batch, Channel, Height, Width]   
            spike_input = spike_input.view(spike_input.size(0), spike_input.size(1), -1)
            
            spike_input = self.layers(spike_input)

            spike_input = spike_input.sum(axis=0)

            return spike_input
        










    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    if pre_trained == False:
        if (convTrue_fcFalse == False):
            net = MY_SNN_FC(cfg, inc = synapse_conv_in_channels).to(device)
        else:
            net = MY_SNN_CONV(cfg, inc = synapse_conv_in_channels).to(device)
        net = torch.nn.DataParallel(net)
    else:
        net = torch.load(pre_trained_path)

    val_acc = 0

    net = net.to(device)

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    # optimizer = torch.optim.Adam(net.parameters(), lr=0.00001)


    for epoch in range(epoch_num):
        print('epoch', epoch)
        epoch_start_time = time.time()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            net.train()

            # print('\niter', i)
            iter_one_train_time_start = time.time()

            inputs, labels = data

            if rate_coding == True :
                inputs = spikegen.rate(inputs, num_steps=TIME)
            else :
                inputs = inputs.repeat(TIME, 1, 1, 1, 1)
            
            # inputs: [Time, Batch, Channel, Height, Width]   

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # inputs: [Time, Batch, Channel, Height, Width]   
            inputs = inputs.permute(1, 0, 2, 3, 4) # net에 넣어줄때는 batch가 젤 앞 차원으로 와야함.
            # inputs: [Batch, Time, Channel, Height, Width]   


            outputs = net(inputs)

            batch = BATCH 
            if labels.size(0) != BATCH: 
                batch = labels.size(0)


            ####### training accruacy ######
            correct = 0
            total = 0
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted[0:batch] == labels).sum().item()
            if i % verbose_interval == 9:
                print(f'training acc: {100 * correct / total:.2f}%')
            ################################

            loss = criterion(outputs[0:batch,:], labels)
            loss.backward()

            # optimizer.zero_grad()와 loss.backward() 호출 후에 실행해야 합니다.
            # if (i % 100 == 9):
            #     print('\n\nepoch', epoch, 'iter', i)
            #     for name, param in net.named_parameters():
            #         if param.requires_grad:
            #             print('\n\n\n\n' , name, param.grad)
            
            optimizer.step()

            running_loss += loss.item()
            # print("Epoch: {}, Iter: {}, Loss: {}".format(epoch + 1, i + 1, running_loss / 100))

            iter_one_train_time_end = time.time()
            elapsed_time = iter_one_train_time_end - iter_one_train_time_start  # 실행 시간 계산
            # print(f"iter_one_train_time: {elapsed_time} seconds")


            if i % verbose_interval == 9:
                iter_one_val_time_start = time.time()
                
                correct = 0
                total = 0
                with torch.no_grad():
                    net.eval()
                    how_many_val_image=0
                    for data in test_loader:
                        how_many_val_image += 1
                        inputs, labels = data
            
                        if rate_coding == True :
                            inputs = spikegen.rate(inputs, num_steps=TIME)
                        else :
                            inputs = inputs.repeat(TIME, 1, 1, 1, 1)

                        
                        inputs = inputs.to(device)
                        labels = labels.to(device)
                        outputs = net(inputs.permute(1, 0, 2, 3, 4))
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        batch = BATCH 
                        if labels.size(0) != BATCH: 
                            batch = labels.size(0)
                        correct += (predicted[0:batch] == labels).sum().item()
                    print(f'validation acc: {100 * correct / total:.2f}%')


                iter_one_val_time_end = time.time()
                elapsed_time = iter_one_val_time_end - iter_one_val_time_start  # 실행 시간 계산
                print(f"iter_one_val_time: {elapsed_time} seconds")
                if val_acc < correct / total:
                    val_acc = correct / total
                    torch.save(net.state_dict(), "net_save/save_now_net_weights.pth")
                    torch.save(net, "net_save/save_now_net.pth")
                    torch.save(net.module.state_dict(), "net_save/save_now_net_weights2.pth")
                    torch.save(net.module, "net_save/save_now_net2.pth")
        epoch_time_end = time.time()
        epoch_time = epoch_time_end - epoch_start_time  # 실행 시간 계산
        
        print(f"epoch_time: {epoch_time} seconds")
        print('\n')


In [ ]:

    cfg = {
    'A': [64, 64], 
    'B': [64, 64, 64, 64], 
    'C': [64, 128, 256],
    'D': [64, 128, 'P', 256, 256, 'P', 512, 512, 'P', 512, 512],
    'K': [64, 64],
    }